# Pix2Pix : Aerial images to maps

Work based on : "**Image-to-Image Translation with Conditional Adversarial Networks**" 

(See [arXiv:1611.07004v3 [cs.CV]](https://arxiv.org/abs/1611.07004) by Phillip Isola, Jun-Yan Zhu, Tinghui Zhou and Alexei A. Efros - [Project Homepage](https://github.com/phillipi/pix2pix) )


(NEW) See also https://www.tensorflow.org/alpha/tutorials/generative/pix2pix in Tensorflow 2.0

-------------------------
REMOTE : Deploy and run a Kubeflow Pipeline from outside the Kubeflow cluster
-------------------------

#### Prerequisites for this demo:
- a Google Cloud Platform (GCP) project with a **IAP-enabled cluster** running on Kubernetes Engine (GKE)
- a GCP service account with the necessary permissons, and added as an 'IAP-secured Web App User'

Some instructions on how to setup this area available [here](https://github.com/amygdala/examples/blob/cookbook/cookbook/pipelines/notebooks/kfp_remote_deploy.ipynb)

In summary, you have to define in your GCP/GKE environment, a Google json key file for the service account deployment on the GKE Kubeflow cluster. To run the notebook locally, you have to set the GOOGLE_APPLICATION_CREDENTIALS environment var to point to your service account credentials:

`export GOOGLE_APPLICATION_CREDENTIALS=<your_json_key_file_path> `

(Note : you should do this before launching your Jupyter Notebook server)

**NOTE** We will reuse *the same* Python code we have used to Build a Pix2Pix

In [1]:
# Check the Google JSON key file for the service account deployment on the GKE Kubeflow cluster
!echo $GOOGLE_APPLICATION_CREDENTIALS

#!cat $GOOGLE_APPLICATION_CREDENTIALS   # It's a secret !


/media/shared/others/pix2pix-map/nextatos-201903-1ef27dfd68c6.json


#### Get Ready for notebook execution


In [2]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

import os

# Import PiX2Pix code 
from utils import *
from download_dataset import *
from prepare_dataset import *
from train_pix2pix import *

#### Kubeflow SDK setup

In [3]:
# Import the Kubeflow Pipelines SDK
import kfp
import kfp.dsl as dsl
import kfp.gcp as gcp
import kfp.notebook
import kfp.components as comp
from kfp import compiler
from kubernetes import client as k8s_client

#### Step 1 : Convert Python Functions into Pipeline operations

In [4]:
#--------------------------------------------- 
# Convert Python Functions into Pipeline operations 
#---------------------------------------------

download_op = comp.func_to_container_op(download_dataset,
                                        base_image='tensorflow/tensorflow:1.12.0-py3' )


preparation_op = comp.func_to_container_op(prepare_dataset,
                                           base_image='tensorflow/tensorflow:1.12.0-py3' )

# Training Component will be executed on a CPU node in this quick demo test
training_op = comp.func_to_container_op(train_pix2pix,
                                       base_image='tensorflow/tensorflow:1.12.0-gpu-py3' )


#### Step 2 : Build and Compile the pix2pix Pipeline Function

In [5]:
URL = "https://people.eecs.berkeley.edu/~tinghuiz/projects/pix2pix/datasets/maps.tar.gz"
FILE_NAME = "maps.tar.gz"

#--------------------------------------------- 
#           NFS PATHS on GKE
#---------------------------------------------
NFS_MOUNT = "/mnt/nfs"
KERAS_CACHE_DIR = "/mnt/nfs/data/"
PATH_TO_TFRECORDS = "/mnt/nfs/data/datasets/{{workflow.name}}"  
PATH_TO_OUTPUTS = "/mnt/nfs/data/outputs/{{workflow.name}}"
PATH_TO_CHECKPOINTS ="/mnt/nfs/data/models/{{workflow.name}}"

#--------------------------------------------- 
#              Google Storage 
#
# (so that Kubeflow pipelines can display 
#  logs them in the Tensorboard widget)
#
# IMPORTANT  : CUSTOMIZE this variable with 
#              your own Bucket URL
#
#--------------------------------------------- 
PATH_TO_TF_LOGS = 'gs://wl-tex10-kfp-001/tf-logs/{{workflow.name}}'


#--------------------------------------------- 
#      Build the pix2pix Pipeline Function
#---------------------------------------------

@dsl.pipeline(
    name='Pix2Pix pipeline',
    description='A pipeline to download and prepare the dataset and train Pix2Pix'
)
def pix2pix(
    
    ## -- Download Dataset Kubeflow Pipeline component parameters (with default values)
    origin = dsl.PipelineParam('origin', value=URL),
    fname = dsl.PipelineParam('fname', value=FILE_NAME),
    cachedir = dsl.PipelineParam('cachedir', value=KERAS_CACHE_DIR), # on Kubeflow GKE/NFS 
    cachesubdir = dsl.PipelineParam('cachesubdir', value="datasets"),
    
    ## -- Prepare Dataset Kubeflow Pipeline component parameters (with default values)
    pathimgsubdir = dsl.PipelineParam('pathimgsubdir', value="train/"),
    pathtfrecords = dsl.PipelineParam('pathtfrecords', value=PATH_TO_TFRECORDS), # on Kubeflow GKE/NFS
    
    # earlystop param will be used in several pipelines components  
    earlystop = dsl.PipelineParam('earlystop', value=10),
    
    ## -- Training Kubeflow Pipeline component (with default values)
    pathtflogs = dsl.PipelineParam('pathtflogs', value=PATH_TO_TF_LOGS),
    pathoutputs = dsl.PipelineParam('pathoutputs', value=PATH_TO_OUTPUTS),
    pathcheckpoints = dsl.PipelineParam('pathcheckpoints', value=PATH_TO_CHECKPOINTS),
    epochs = dsl.PipelineParam('epochs', value="1"), 
    initialresize = dsl.PipelineParam('initialresize', value="286"), 
    cropresize = dsl.PipelineParam('cropresize', value="256"),
    resizemethod = dsl.PipelineParam('resizemethod', value="1"), 
    saveevery = dsl.PipelineParam('saveevery', value="1")
   
):
    
       
    # Passing pipeline parameters as operation arguments (Returns a dsl.ContainerOp class instance)
    download_task = download_op(fname, origin, cachedir, cachesubdir) \
                                .add_volume(k8s_client.V1Volume(name='workdir', 
                                                                persistent_volume_claim=k8s_client.V1PersistentVolumeClaimVolumeSource(claim_name='nfs'))) \
                                .add_volume_mount(k8s_client.V1VolumeMount(mount_path=NFS_MOUNT, name='workdir'))
    
    
    # Single output value of previous pipeline component will be used in input of next pipeline component 
    preparation_task = preparation_op(download_task.output, pathimgsubdir, pathtfrecords, earlystop ) \
                                      .add_volume(k8s_client.V1Volume(name='workdir', 
                                                                      persistent_volume_claim=k8s_client.V1PersistentVolumeClaimVolumeSource(claim_name='nfs'))) \
                                      .add_volume_mount(k8s_client.V1VolumeMount(mount_path=NFS_MOUNT, name='workdir'))
    
    
    # Single output value of previous pipeline component will be used in input of next pipeline component 
    training_task = training_op(preparation_task.output, pathtflogs, pathoutputs, pathcheckpoints, epochs,
                                initialresize, cropresize, resizemethod, saveevery, earlystop) \
                                .add_volume(k8s_client.V1Volume(name='workdir', 
                                                                persistent_volume_claim=k8s_client.V1PersistentVolumeClaimVolumeSource(claim_name='nfs'))) \
                                .add_volume_mount(k8s_client.V1VolumeMount(mount_path=NFS_MOUNT, name='workdir'))
    
    # Allow to write Tensorboard logs on Google Storage
    training_task.apply(gcp.use_gcp_secret('user-gcp-sa')) 
    
    # To the training task on a GPU node
    training_task.set_gpu_limit(1)   
    

#### Step 3 : Create/Reuse an *EXPERIMENT* and submit a Pipeline *RUN*

In [6]:
#--------------------------------------------- 
#            Compile the Pipeline 
#--------------------------------------------- 
pipeline_filename = pix2pix.__name__ + '.pipeline.tar.gz'
compiler.Compiler().compile(pipeline_func=pix2pix, 
                            package_path=pipeline_filename)

**(NEW)** Create an instance of the Kubeflow Pipelines Client

The only change to submit a remote execution on the Kubeflow cluster, is to pass additional parameters in instance creation  of the Kubeflow Pipelines.

Just change this:
```
client = kfp.Client()
```
bythis:

```
client = kfp.Client(host=<YOUR_KUPEFLOW_PIPELINE_ENDPOINTS_URL>/pipeline ,
                    client_id=>YOUR_CLIENT_ID)
```                    
                    
*Et Voilà!*

In [7]:
#--------------------------------------------- 
#  Create a Kubeflow Pipeline Experiment
#--------------------------------------------- 
#---------------------------------------------------- 
EXPERIMENT_NAME = "Next - Pix2Pix"   ## Customize Name

#--------------------------------------------- 
#         Create an instance of the 
#         Kubeflow Pipelines client
#
# IMPORTANT  : CUSTOMIZE this variable  
#             with your own cluster info
#
#--------------------------------------------- 
client = kfp.Client(host=<YOUR_KUPEFLOW_PIPELINE_ENDPOINTS_URL>/pipeline ,
                    client_id=>YOUR_CLIENT_ID)


In [8]:
try:
    experiment = client.get_experiment(experiment_name=EXPERIMENT_NAME)
except:
    experiment = client.create_experiment(EXPERIMENT_NAME)

#-------------------------------------------------- 
#  Optional : Specify/Overwrite pipeline arguments 
#  values for execution or use default values)
#-------------------------------------------------- 
arguments = {'epochs': 20, # Change to 200 for a full training
             'initialresize' : 286,
             'cropresize': 256,
             'saveevery': 100,
             'earlystop': 0  
            }

#-------------------------------------------------- 
#             Submit a pipeline run
#--------------------------------------------------
run_name = pix2pix.__name__ + ' remote run'
#run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename)
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)

--------------------------------------------------
#  Don't care about the environment thanks to GKE Autoscaling !

Let's Submit another pipeline run using a GPU...and let's see what is happening on the GCP console

In [ ]:
#-------------------------------------------------- 
#  Optional : Specify/Overwrite pipeline arguments 
#  values for execution or use default values)
#-------------------------------------------------- 
arguments = {'epochs': 20, # Change to 200 for a full training
             'initialresize' : 286,
             'cropresize': 256,
             'saveevery': 100,
             'earlystop': 0  
            }

#-------------------------------------------------- 
#             Submit a pipeline run
#--------------------------------------------------
run_name = pix2pix.__name__ + ' - Remote run 2 (with Autoscaling)'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)